In [2]:
import dash
#pip install dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
#!pip install keras
#import necessary libraries
from plotly import tools
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go #for calling graph objects from plotly library.
import pandas as pd
import numpy as np 
import requests
#!pip install wget
#!pip install plotly-geo
from datetime import datetime
from scipy.interpolate import interp1d
import os

import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import random
import math
import time
#import xgboost

# from sklearn.linear_model import LinearRegression, BayesianRidge
# from sklearn.model_selection import RandomizedSearchCV, train_test_split
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import operator 

#plt.style.use('fivethirtyeight')
%matplotlib inline

In [39]:
#usa data 
confirmed_df_us =  pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
death_df_us = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [40]:
#transpose all date columns into values (use melt to unpivot dataframes from wide to long format)
dates_us = confirmed_df_us.columns[12:]
confirmed_df_long_us = confirmed_df_us.melt(
    id_vars=['Province_State',  "FIPS", 'Lat', 'Long_'], 
    value_vars=dates_us, 
    var_name='Date', 
    value_name='Confirmed'
)
death_df_long_us = death_df_us.melt(
    id_vars=['Province_State', "FIPS", 'Lat', 'Long_'], 
    value_vars=dates_us, 
    var_name='Date', 
    value_name='Deaths'
)

In [41]:
#Merging confirmed_df_long and deaths_df_long
full_merge_us = confirmed_df_long_us.merge(
  right=death_df_long_us, 
  how='left',
  on=['Province_State', 'FIPS', 'Date', 'Lat', 'Long_']
)

In [42]:
full_merge_us['LogConfirmed'] = np.log(full_merge_us['Confirmed'])
full_merge_us['LogDeaths'] = np.log(full_merge_us['Deaths'])

/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log



In [43]:
import datetime

full_merge_us["Date"] = pd.to_datetime(full_merge_us["Date"], errors='coerce')
#df['DATE'] = pd.to_datetime(df['Date'], format= "%d-%b-%y %H:%M:%S")
cumulative_df_us = full_merge_us.groupby(["Province_State","Date", "Lat", "Long_"])["Confirmed","Deaths"].sum().reset_index()
cumulative_df_us["Date"] = pd.to_datetime(cumulative_df_us["Date"] , format="%mm/%dd/%YY").dt.date
cumulative_df_us = cumulative_df_us.sort_values(by="Date").reset_index(drop=True)
start_date = datetime.date(2020, 2, 25)
cumulative_df_us = cumulative_df_us[cumulative_df_us["Date"]>=start_date]
cumulative_df_us["Date"] = cumulative_df_us["Date"].astype(str)


/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [45]:
cumulative_df_us['Date'] = pd.to_datetime(cumulative_df_us['Date'])

In [46]:
us_grouped = cumulative_df_us.groupby(['Date', 'Province_State', 'Lat', 'Long_'])['Confirmed', 'Deaths'].sum().reset_index()

/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [47]:
us_grouped

,Date,Province_State,Lat,Long_,Confirmed,Deaths
0,2020-02-25,Alabama,0.000000,0.000000,0,0
1,2020-02-25,Alabama,30.727750,-87.722071,0,0
2,2020-02-25,Alabama,30.784723,-88.208424,0,0
3,2020-02-25,Alabama,31.093890,-85.835728,0,0
4,2020-02-25,Alabama,31.125679,-87.159187,0,0
...,...,...,...,...,...,...
451336,2020-07-14,Wyoming,44.248861,-105.547440,89,0
451337,2020-07-14,Wyoming,44.521575,-109.585282,84,0
451338,2020-07-14,Wyoming,44.524051,-107.996037,30,0
451339,2020-07-14,Wyoming,44.588551,-104.569770,8,0


In [48]:
#add day wise New cases, New deaths and New recovered 
# new cases 
temp = us_grouped.groupby(['Province_State', 'Date', ])['Confirmed', 'Deaths']
temp = temp.sum().diff().reset_index()
mask = temp['Province_State'] != temp['Province_State'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan

# renaming columns
temp.columns = ['Province_State', 'Date', 'New cases', 'New deaths']
# merging new values
us_grouped = pd.merge(us_grouped, temp, on=['Province_State', 'Date'])
# filling na with 0
us_grouped = us_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths']
us_grouped[cols] = us_grouped[cols].astype('int')
# 
us_grouped['New cases'] = us_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [68]:
us_grouped.groupby(['Date', 'Province_State', 'Lat', 'Long_'])['Confirmed', 'Deaths', 'New cases', 'New deaths'].sum().reset_index()

/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Date,Province_State,Lat,Long_,Confirmed,Deaths,New cases,New deaths
0,2020-02-25,Alabama,0.000000,0.000000,0,0,0,0
1,2020-02-25,Alabama,30.727750,-87.722071,0,0,0,0
2,2020-02-25,Alabama,30.784723,-88.208424,0,0,0,0
3,2020-02-25,Alabama,31.093890,-85.835728,0,0,0,0
4,2020-02-25,Alabama,31.125679,-87.159187,0,0,0,0
...,...,...,...,...,...,...,...,...
451336,2020-07-14,Wyoming,44.248861,-105.547440,89,0,47,1
451337,2020-07-14,Wyoming,44.521575,-109.585282,84,0,47,1
451338,2020-07-14,Wyoming,44.524051,-107.996037,30,0,47,1
451339,2020-07-14,Wyoming,44.588551,-104.569770,8,0,47,1


In [70]:
#opsd_daily = pd.read_csv('opsd_germany_daily.csv', index_col=0, parse_dates=True)

us_df=pd.read_csv('US-COVID-19-time-series-clean-complete.csv', index_col=0, parse_dates=True)
us_df.groupby(['Date', 'Province_State', 'Lat', 'Long_'])['Confirmed', 'Deaths', 'New cases', 'New deaths'].sum().reset_index()

/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Date,Province_State,Lat,Long_,Confirmed,Deaths,New cases,New deaths
0,2020-02-25,Alabama,0.000000,0.000000,0,0,0,0
1,2020-02-25,Alabama,30.727750,-87.722071,0,0,0,0
2,2020-02-25,Alabama,30.784723,-88.208424,0,0,0,0
3,2020-02-25,Alabama,31.093890,-85.835728,0,0,0,0
4,2020-02-25,Alabama,31.125679,-87.159187,0,0,0,0
...,...,...,...,...,...,...,...,...
451336,2020-07-14,Wyoming,44.248861,-105.547440,89,0,47,1
451337,2020-07-14,Wyoming,44.521575,-109.585283,84,0,47,1
451338,2020-07-14,Wyoming,44.524051,-107.996037,30,0,47,1
451339,2020-07-14,Wyoming,44.588551,-104.569771,8,0,47,1


In [73]:
us_df.loc[us_df.index!='Confirmed'].sum()

/Users/biancahernandez/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:122: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Date              2020-02-252020-02-252020-02-252020-02-252020-0...
Province_State    AlabamaAlabamaAlabamaAlabamaAlabamaAlabamaAlab...
Lat                                                     1.70493e+07
Long_                                                  -4.09008e+07
Confirmed                                                 173868472
Deaths                                                      9163312
New cases                                                 280749957
New deaths                                                  8852776
dtype: object

In [ ]:
fig = px.scatter_mapbox(cumulative_df_us, 
                        lat="Lat", 
                        lon="Long_", 
                        size=cumulative_df_us["Confirmed"]**.5*100,
                        template = "seaborn",
                        color = "Confirmed",  
                        color_continuous_scale="spectral",
                        zoom=2, 
                        size_max=10, 
                        animation_frame="Date",
                        hover_name="Province_State", 
                        hover_data= ["Confirmed"])
                        
                            

fig.update_layout(margin=dict(l=1, r=1, t=1, b=1),
                  autosize=True,
                  mapbox_style="carto-positron")
#update different layouts
fig.layout.sliders[0].currentvalue.prefix=""
fig.layout.sliders[0].len=.9
fig.layout.sliders[0].currentvalue.font.color="black"
fig.layout.sliders[0].currentvalue.font.size=18
#fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200
fig.show()

In [ ]:
#slider with moving average for practice in widgets
#packages
import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cumulative_df_us.head()

In [ ]:
# create a list to store all the state names
states = df_comb_sum.state.unique()
import plotly.graph_objects as go
import ipywidgets
from ipywidgets import widgets
@ipywidgets.interact
def plot(state= widgets.Dropdown(options=states,value="New York", description='State:',disabled=False,), \
         ma_window=widgets.IntSlider(min=3, max=15, step=1, value=7, description="SMA window:")):           
    
    df['new_cases_sma'] = df['new_confirmed'].rolling(window=ma_window).mean()
fig = go.Figure()
fig.add_trace(go.Scatter(
        x=df.index,
        y=df['new_confirmed'],
        mode="lines",
        name="Daily New Confirmed Cases"
    ))
fig.add_trace(go.Scatter(
        x=df.index,
        y=df['new_cases_sma'],
        mode="lines", 
        name= "Daily New Cases SMA{} ".format(ma_window)
    ))
 
fig.show()